## Step 1: Load and Explore Data

In [1]:
import pandas as pd
import numpy as np
from typing import List, Dict, Any
import os

# Load the Wharton data
file_path = 'WhartonDataSource.xlsx'
df = pd.read_excel(file_path)

print(f"Data shape: {df.shape}")
print(f"\nColumns: {df.columns.tolist()}")
print(f"\nFirst few rows:")
df.head()

Data shape: (1048575, 76)

Columns: ['(tic) Ticker Symbol', '(datadate) Data Date - Dividends', '(conm) Company Name', '(cusip) CUSIP', '(cik) CIK Number', '(exchg) Stock Exchange Code', '(secstat) Security Status Marker', '(fic) Current ISO Country Code - Incorporation', '(tpci) Issue Type Code', '(add1) Address Line 1', '(add2) Address Line 2', '(add3) Address Line 3', '(add4) Address Line 4', '(addzip) Postal Code', '(busdesc) S&P Business Description', '(city) City', '(conml) Company Legal Name', '(costat) Active/Inactive Status Marker', '(county) County Code', '(dldte) Research Company Deletion Date', '(dlrsn) Research Co Reason for Deletion', '(ein) Employer Identification Number', '(fax) Fax Number', '(fyrc) Current Fiscal Year End Month', '(ggroup) GIC Groups', '(gind) GIC Industries', '(gsector) GIC Sectors', '(gsubind) GIC Sub-Industries', '(idbflag) International, Domestic, Both Indicator', '(incorp) Current State/Province of Incorporation Code', '(ipodate) Company Initial P

,(tic) Ticker Symbol,(datadate) Data Date - Dividends,(conm) Company Name,(cusip) CUSIP,(cik) CIK Number,(exchg) Stock Exchange Code,(secstat) Security Status Marker,(fic) Current ISO Country Code - Incorporation,(tpci) Issue Type Code,(add1) Address Line 1,...,(paydate) Dividend Payment Date,(paydateind) Dividend Payment Date Indicator,(prccd) Price - Close - Daily,(prchd) Price - High - Daily,(prcld) Price - Low - Daily,(prcod) Price - Open - Daily,(prcstd) Price Status Code - Daily,(recorddate) Dividend Record Date,(trfd) Daily Total Return Factor,(gvkey) Global Company Key - Dividends
0,AAL,2013-12-09,AMERICAN AIRLINES GROUP INC,02376R102,6201,14,A,USA,0,1 Skyview Drive,...,NaT,NaN,24.60,25.44,23.4501,23.95,3.0,NaT,1.0,1045
1,AAL,2013-12-10,AMERICAN AIRLINES GROUP INC,02376R102,6201,14,A,USA,0,1 Skyview Drive,...,NaT,NaN,24.88,25.17,24.4100,24.54,3.0,NaT,1.0,1045
2,AAL,2013-12-11,AMERICAN AIRLINES GROUP INC,02376R102,6201,14,A,USA,0,1 Skyview Drive,...,NaT,NaN,25.99,27.20,25.3700,25.44,3.0,NaT,1.0,1045
3,AAL,2013-12-12,AMERICAN AIRLINES GROUP INC,02376R102,6201,14,A,USA,0,1 Skyview Drive,...,NaT,NaN,25.45,26.71,25.4500,26.20,3.0,NaT,1.0,1045
4,AAL,2013-12-13,AMERICAN AIRLINES GROUP INC,02376R102,6201,14,A,USA,0,1 Skyview Drive,...,NaT,NaN,26.23,26.30,25.5201,25.61,3.0,NaT,1.0,1045


In [ ]:
def extract_and_rename_columns(df):
    """
    Extract shortened column names from parentheses in full column names.
    
    Parameters
    ----------
    df : pd.DataFrame
        DataFrame with column names in format: (SHORT_NAME) Full Name Description
    
    Returns
    -------
    tuple
        (modified_df, column_mapping)
        - modified_df: DataFrame with columns renamed to shortened names
        - column_mapping: Dict mapping short names to full descriptions
    """
    column_mapping = {}
    rename_dict = {}
    
    for col in df.columns:
        if col.startswith('(') and ')' in col:
            short_name = col.split(')')[0][1:]  # Extract text inside parentheses
            full_name = col.split(')')[1].strip()  # Extract text after parentheses
            column_mapping[short_name] = full_name
            rename_dict[col] = short_name
    
    # Print the mapping for reference
    print("Column mapping (shortened name -> full name):")
    for short, full in column_mapping.items():
        print(f"  {short:15} -> {full}")
    
    # Rename columns in the dataframe
    df_renamed = df.rename(columns=rename_dict)
    
    return df_renamed, column_mapping

# Call the function
df, col_mapping = extract_and_rename_columns(df)
print(f"\nRenamed {len(col_mapping)} columns successfully!")

Column mapping (shortened name -> full name):
tic -> Ticker Symbol
datadate -> Data Date - Dividends
conm -> Company Name
cusip -> CUSIP
cik -> CIK Number
exchg -> Stock Exchange Code
secstat -> Security Status Marker
fic -> Current ISO Country Code - Incorporation
tpci -> Issue Type Code
add1 -> Address Line 1
add2 -> Address Line 2
add3 -> Address Line 3
add4 -> Address Line 4
addzip -> Postal Code
busdesc -> S&P Business Description
city -> City
conml -> Company Legal Name
costat -> Active/Inactive Status Marker
county -> County Code
dldte -> Research Company Deletion Date
dlrsn -> Research Co Reason for Deletion
ein -> Employer Identification Number
fax -> Fax Number
fyrc -> Current Fiscal Year End Month
ggroup -> GIC Groups
gind -> GIC Industries
gsector -> GIC Sectors
gsubind -> GIC Sub-Industries
idbflag -> International, Domestic, Both Indicator
incorp -> Current State/Province of Incorporation Code
ipodate -> Company Initial Public Offering Date
loc -> Current ISO Country Code

In [12]:
# Inspect key columns
print("Key column info:")
key_cols = ['tic', 'datadate', 'prccd', 'ajexdi', 'cshtrd', 'trfd']
available_cols = [col for col in key_cols if col in df.columns]

print(f"Available key columns: {available_cols}")
print(f"\nData types:")
print(df[available_cols].dtypes)
print(f"\nMissing values:")
print(df[available_cols].isnull().sum())
print(f"\nSample data:")
df[available_cols].head(10)

Key column info:
Available key columns: ['tic', 'datadate', 'prccd', 'ajexdi', 'cshtrd', 'trfd']

Data types:
tic                 object
datadate    datetime64[ns]
prccd              float64
ajexdi             float64
cshtrd             float64
trfd               float64
dtype: object

Missing values:
tic             0
datadate        0
prccd          17
ajexdi         17
cshtrd         25
trfd        26100
dtype: int64

Sample data:


,tic,datadate,prccd,ajexdi,cshtrd,trfd
0,AAL,2013-12-09,24.60,1.0,43167060.0,1.0
1,AAL,2013-12-10,24.88,1.0,18648140.0,1.0
2,AAL,2013-12-11,25.99,1.0,38584270.0,1.0
3,AAL,2013-12-12,25.45,1.0,19977100.0,1.0
4,AAL,2013-12-13,26.23,1.0,12189890.0,1.0
5,AAL,2013-12-16,26.61,1.0,13181320.0,1.0
6,AAL,2013-12-17,26.10,1.0,11398040.0,1.0
7,AAL,2013-12-18,26.23,1.0,9989747.0,1.0
8,AAL,2013-12-19,26.12,1.0,6908812.0,1.0
9,AAL,2013-12-20,26.33,1.0,7527964.0,1.0


In [13]:
# Check date range and unique tickers
df['(datadate) Data Date - Dividends'] = pd.to_datetime(df['datadate'])

print(f"Date range: {df['datadate'].min()} to {df['datadate'].max()}")
print(f"Number of unique tickers: {df['tic'].nunique()}")
print(f"\nTop 10 tickers by number of records:")
print(df['tic'].value_counts().head(10))

Date range: 2000-01-03 00:00:00 to 2025-12-10 00:00:00
Number of unique tickers: 167

Top 10 tickers by number of records:
tic
LNT     6526
MO      6526
PSA     6526
HBAN    6526
TXN     6526
ADP     6526
TXT     6526
JCI     6526
MS      6526
RF      6526
Name: count, dtype: int64


## Step 2: Create WhartonDataSource Class

In [14]:
from abc import ABC, abstractmethod

class DataSource(ABC):
    """Interface for fetching historical market data."""
    
    @abstractmethod
    def get_historical_data(self, tickers: List[str], start_date: str, end_date: str) -> pd.DataFrame:
        """Fetch historical data for given tickers and date range."""
        pass

class WhartonDataSource(DataSource):
    """
    Implementation of DataSource using Wharton WRDS data from Excel file.
    
    Expected columns:
    - tic: Ticker Symbol
    - datadate: Data Date
    - prccd: Price Close Daily
    - ajexdi: Adjustment Factor (cumulative by ex-date)
    - cshtrd: Trading Volume Daily
    - trfd: Daily Total Return Factor (optional, alternative to prccd*ajexdi)
    """
    
    def __init__(self, file_path: str, use_trfd: bool = False):
        """
        Parameters
        ----------
        file_path : str
            Path to the Wharton Excel/CSV file
        use_trfd : bool
            If True, use trfd (total return factor) directly.
            If False, calculate adjusted price as prccd * ajexdi
        """
        self.file_path = file_path
        self.use_trfd = use_trfd
        self.data = None
        self._load_data()
    
    def _load_data(self):
        """Load and prepare data from file."""
        if not os.path.exists(self.file_path):
            raise FileNotFoundError(f"Data file not found at {self.file_path}")
        
        # Load file (supports .xlsx and .csv)
        if self.file_path.endswith('.xlsx') or self.file_path.endswith('.xls'):
            self.data = pd.read_excel(self.file_path)
        else:
            self.data = pd.read_csv(self.file_path)
        
        # Ensure required columns exist
        required_cols = ['tic', 'datadate']
        missing_cols = [col for col in required_cols if col not in self.data.columns]
        if missing_cols:
            raise ValueError(f"Missing required columns: {missing_cols}")
        
        # Convert date column
        self.data['datadate'] = pd.to_datetime(self.data['datadate'])
        
        # Calculate adjusted close price
        if self.use_trfd:
            if 'trfd' not in self.data.columns:
                raise ValueError("trfd column not found but use_trfd=True")
            self.data['Adj Close'] = self.data['trfd']
        else:
            if 'prccd' not in self.data.columns or 'ajexdi' not in self.data.columns:
                raise ValueError("prccd and ajexdi columns required when use_trfd=False")
            self.data['Adj Close'] = self.data['prccd'] * self.data['ajexdi']
        
        # Add volume if available
        if 'cshtrd' in self.data.columns:
            self.data['Volume'] = self.data['cshtrd']
        
        print(f"Loaded {len(self.data)} records for {self.data['tic'].nunique()} tickers")
        print(f"Date range: {self.data['datadate'].min()} to {self.data['datadate'].max()}")
    
    def get_historical_data(self, tickers: List[str], start_date: str, end_date: str) -> pd.DataFrame:
        """
        Fetch historical adjusted prices for given tickers and date range.
        
        Returns
        -------
        pd.DataFrame
            Index: dates, Columns: tickers, Values: adjusted close prices
        """
        start_ts = pd.Timestamp(start_date)
        end_ts = pd.Timestamp(end_date)
        
        # Filter by date range and tickers
        mask = (
            (self.data['datadate'] >= start_ts) & 
            (self.data['datadate'] <= end_ts) & 
            (self.data['tic'].isin(tickers))
        )
        filtered = self.data.loc[mask, ['tic', 'datadate', 'Adj Close']].copy()
        
        if filtered.empty:
            print(f"Warning: No data found for specified tickers and date range")
            return pd.DataFrame()
        
        # Pivot to get tickers as columns, dates as index
        result = filtered.pivot(index='datadate', columns='tic', values='Adj Close')
        result = result.sort_index()
        
        # Report missing tickers
        missing_tickers = set(tickers) - set(result.columns)
        if missing_tickers:
            print(f"Warning: No data found for tickers: {missing_tickers}")
        
        return result
    
    def get_historical_data_with_volume(self, tickers: List[str], start_date: str, end_date: str) -> Dict[str, pd.DataFrame]:
        """
        Fetch historical price and volume data, organized by ticker.
        
        Returns
        -------
        Dict[str, pd.DataFrame]
            For each ticker: DataFrame indexed by date with 'Adj Close' and 'Volume' columns
        """
        start_ts = pd.Timestamp(start_date)
        end_ts = pd.Timestamp(end_date)
        
        result = {}
        
        for ticker in tickers:
            mask = (
                (self.data['datadate'] >= start_ts) & 
                (self.data['datadate'] <= end_ts) & 
                (self.data['tic'] == ticker)
            )
            ticker_data = self.data.loc[mask, ['datadate', 'Adj Close', 'Volume']].copy()
            
            if ticker_data.empty:
                print(f"Warning: No data found for {ticker}")
                continue
            
            ticker_data = ticker_data.set_index('datadate').sort_index()
            result[ticker] = ticker_data
        
        return result

print("WhartonDataSource class created successfully!")

WhartonDataSource class created successfully!


## Step 3: Test Basic Functionality

In [15]:
# Initialize the data source
wharton_source = WhartonDataSource('WhartonDataSource.xlsx', use_trfd=False)

# Test with a few tickers
test_tickers = ['AAPL', 'MSFT', 'GOOGL', 'SPY']
start_date = '2024-01-01'
end_date = '2024-12-31'

price_data = wharton_source.get_historical_data(test_tickers, start_date, end_date)
print(f"\nRetrieved data shape: {price_data.shape}")
print(f"\nFirst few rows:")
price_data.head()

ValueError: Missing required columns: ['tic', 'datadate']

In [ ]:
# Check for missing data
print("Missing data per ticker:")
print(price_data.isnull().sum())
print(f"\nData completeness: {(1 - price_data.isnull().sum() / len(price_data)) * 100}%")

## Step 4: Test with SPY Verification (If Holdings File Available)

In [ ]:
# Import necessary functions from data_source module
import sys
sys.path.append('/Users/bouse/developer/CDS/DataQuality25/millennium-data-quality-25-26/backtester')
from data_source import YahooFinanceDataSource

# Check if holdings file exists
holdings_file = 'holdings-daily-us-en-spy.xlsx'
if os.path.exists(holdings_file):
    print("Holdings file found! Running SPY verification...")
    
    # Use YahooFinanceDataSource methods for holdings and portfolio calculation
    yahoo_source = YahooFinanceDataSource()
    holdings_df = yahoo_source.read_spy_holdings(holdings_file)
    
    # Fix ticker format
    holdings_df['Ticker'] = holdings_df['Ticker'].replace({
        'BRK.B': 'BRK-B',
        'BF.B': 'BF-B',
        '-': 'USD'
    })
    
    print(f"Found {len(holdings_df)} constituents")
    
    # Get data for SPY and constituents
    tickers = ['SPY'] + holdings_df['Ticker'].tolist()
    price_data = wharton_source.get_historical_data(tickers, start_date, end_date)
    
    if 'SPY' in price_data.columns:
        spy_data = price_data['SPY'].copy()
        weighted_portfolio = yahoo_source.calculate_weighted_portfolio(holdings_df, price_data)
        
        print(f"\nSPY data points: {len(spy_data)}")
        print(f"Portfolio data points: {len(weighted_portfolio)}")
    else:
        print("SPY data not available in Wharton source")
else:
    print(f"Holdings file not found: {holdings_file}")
    print("Skipping SPY verification test")

## Step 5: Compare Wharton vs Yahoo Finance (Optional)

In [ ]:
# Compare a single ticker from both sources
import matplotlib.pyplot as plt

test_ticker = 'AAPL'
compare_start = '2024-01-01'
compare_end = '2024-03-31'

# Get Wharton data
wharton_prices = wharton_source.get_historical_data([test_ticker], compare_start, compare_end)

# Get Yahoo data for comparison (if available)
try:
    import yfinance as yf
    yahoo_data = yf.download(test_ticker, start=compare_start, end=compare_end, auto_adjust=False)
    yahoo_prices = yahoo_data['Adj Close']
    
    # Plot comparison
    fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(12, 8))
    
    # Price comparison
    ax1.plot(wharton_prices.index, wharton_prices[test_ticker], label='Wharton', alpha=0.7)
    ax1.plot(yahoo_prices.index, yahoo_prices, label='Yahoo Finance', alpha=0.7)
    ax1.set_title(f'{test_ticker} Price Comparison')
    ax1.set_ylabel('Adjusted Close Price')
    ax1.legend()
    ax1.grid(True)
    
    # Difference
    aligned = pd.concat([wharton_prices[test_ticker], yahoo_prices], axis=1, join='inner')
    aligned.columns = ['Wharton', 'Yahoo']
    aligned['Diff'] = aligned['Wharton'] - aligned['Yahoo']
    aligned['Diff_Pct'] = (aligned['Diff'] / aligned['Yahoo']) * 100
    
    ax2.plot(aligned.index, aligned['Diff_Pct'])
    ax2.set_title('Percentage Difference (Wharton - Yahoo)')
    ax2.set_ylabel('Difference %')
    ax2.set_xlabel('Date')
    ax2.grid(True)
    ax2.axhline(y=0, color='r', linestyle='--', alpha=0.5)
    
    plt.tight_layout()
    plt.show()
    
    print(f"\nDifference Statistics:")
    print(f"Mean difference: {aligned['Diff_Pct'].mean():.4f}%")
    print(f"Max difference: {aligned['Diff_Pct'].max():.4f}%")
    print(f"Correlation: {aligned['Wharton'].corr(aligned['Yahoo']):.6f}")
    
except Exception as e:
    print(f"Could not compare with Yahoo Finance: {e}")

## Next Steps

1. Add `WhartonDataSource` to [data_source.py](data_source.py)
2. Implement additional methods:
   - `read_spy_holdings()` (reuse from YahooFinanceDataSource)
   - `calculate_weighted_portfolio()` (reuse)
   - `detect_price_anomalies()` (reuse)
   - `verify_spy_vs_constituents()` (reuse)
3. Run full backtester tests with Wharton data
4. Compare results with Yahoo Finance data